In [1]:
import os
from scipy.interpolate import interp1d
import pandas as pd
import plotly.express as px
import numpy as np
import geopy

import airbnb_dataset
import config_data

In [17]:
myABData = airbnb_dataset.AirbnbDataset()
ctry_cities = list(config_data.country_to_city.values())
state_cities = []
states = []
for k,v in config_data.US_state_to_city.items():
    state_cities.extend(v)
    for t in range(len(v)):
        states.append(k)

[(0, ('North Carolina', 'Asheville')), (1, ('Texas', 'Austin')), (2, ('Massachusetts', 'Boston')), (3, ('Massachusetts', 'Cambridge')), (4, ('Florida', 'Broward County')), (5, ('Illinois', 'Chicago')), (6, ('Nevada', 'Clark County')), (7, ('Ohio', 'Columbus')), (8, ('Colorado', 'Denver')), (9, ('Hawaii', 'Hawaii')), (10, ('New Jersey', 'Jersey City')), (11, ('California', 'Los Angeles')), (12, ('California', 'Oakland')), (13, ('California', 'Pacific Grove')), (14, ('California', 'San Diego')), (15, ('California', 'San Francisco')), (16, ('California', 'San Mateo County')), (17, ('California', 'Santa Clara County')), (18, ('California', 'Santa Cruz County')), (19, ('Tennessee', 'Nashville')), (20, ('Louisiana', 'New Orleans')), (21, ('New York', 'New York City')), (22, ('Oregon', 'Portland')), (23, ('Oregon', 'Salem')), (24, ('Rhode Island', 'Rhode Island')), (25, ('Washington', 'Seattle')), (26, ('Minnesota', 'Twin Cities MSA')), (27, ('District of Columbia', 'Washington D.C.'))]


In [3]:
#need to process reviews in a datewise manner
# one csv per month, across all locations
alldata=[]
cols = ["city","lat","long"]
for i in range(1,11):
    cols.append("month"+str(i))
print(cols)

['city', 'lat', 'long', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10']


In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
def getlocation(city):
    location = geolocator.geocode(city)
    print(location)
    ns1 = float(location.raw['boundingbox'][0])
    ns2 = float(location.raw['boundingbox'][1])
    ew1 = float(location.raw['boundingbox'][2])
    ew2 = float(location.raw['boundingbox'][3])
    
    return (ns1+ns2)/2,(ew1+ew2)/2

In [13]:
location = geolocator.geocode("Twin Cities Minneapolis")
print(location)

Cedar Lake LRT Regional Trail, Sorenson, Saint Louis Park, Hennepin County, Minnesota, 55416:55426, United States of America


In [22]:
for i,c in enumerate(state_cities):
    df = myABData.get_reviews_for_city(c)
    df19 = df.loc[2019].values[:-2]
    df20 = df.loc[2020].values
    rel_diff = [c]
    if c == 'Twin Cities MSA':
        c = c[:c.rindex(c)]
    
    lat,long = getlocation(c+' ' + states[i])
    rel_diff.append(lat)
    rel_diff.append(long)
    rel_diff.extend(list(((df19 - df20)/df19).T[0]))
    alldata.append(tuple(rel_diff))
    
monthlydf = pd.DataFrame(alldata, columns=cols)
monthlydf = monthlydf.set_index(["city"])
monthlydf.to_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/monthly_reviews.csv'))

City Asheville is in country United States .
Asheville, Buncombe County, North Carolina, United States of America
City Austin is in country United States .
Austin, Travis County, Texas, United States of America
City Boston is in country United States .
Boston, Suffolk County, Massachusetts, United States of America
City Cambridge is in country United States .
Cambridge, Middlesex County, Massachusetts, United States of America
City Broward County is in country United States .
Broward County, Florida, United States of America
City Chicago is in country United States .
Chicago, Cook County, Illinois, United States of America
City Clark County is in country United States .
Clark County, Nevada, United States of America
City Columbus is in country United States .
The Ohio State University, Columbus, Franklin County, 43210, United States of America
City Denver is in country United States .
Denver, Colorado, United States of America
City Hawaii is in country United States .
Hawaiʻi, Hawaii, 

In [40]:
#weekly, 10 months
alldata = []
cols = ["city","lat","long"]

min_n_months = 1000
for i,c in enumerate(state_cities):
    df = myABData.get_reviews_for_city(c,per_week = True,per_month = False)
    df20 = df.loc[2020].values[:41]
    df19 = df.loc[2019].values[:41]
    rel_diff = [c]
    if c == 'Twin Cities MSA':
        c = c[:c.rindex(' ')]
#     if c == 'San Francisco':
#         print(df.loc[2019:2020])
    lat,long = getlocation(c + " "+states[i])
    rel_diff.append(lat)
    rel_diff.append(long)
    rel_diff.extend(list(((df19 - df20)/df19).T[0]))
    alldata.append(tuple(rel_diff))
for i in range(1,42):
    cols.append("week"+str(i))
weeklydf = pd.DataFrame(alldata,columns = cols)
weeklydf.to_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/weekly_reviews.csv'))

City Asheville is in country United States .
Asheville, Buncombe County, North Carolina, United States of America
City Austin is in country United States .
Austin, Travis County, Texas, United States of America
City Boston is in country United States .
Boston, Suffolk County, Massachusetts, United States of America
City Cambridge is in country United States .
Cambridge, Middlesex County, Massachusetts, United States of America
City Broward County is in country United States .
Broward County, Florida, United States of America
City Chicago is in country United States .
Chicago, Cook County, Illinois, United States of America
City Clark County is in country United States .
Clark County, Nevada, United States of America
City Columbus is in country United States .
The Ohio State University, Columbus, Franklin County, 43210, United States of America
City Denver is in country United States .
Denver, Colorado, United States of America
City Hawaii is in country United States .
Hawaiʻi, Hawaii, 

In [69]:
#global
df = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'GLOBAL/weekly_reviews.csv'))
list1 = df.week1.values.tolist()
minval = min(list1)
list1 = list(np.array(list1) - minval)
m = interp1d([0,max(list1)],[1,50])
circ_rad = m(list1)
typelist = ["stamen-watercolor"]

for i in typelist:
    print(i)
    fig = px.density_mapbox(df, lat="lat", lon="long",radius=circ_rad,zoom=0.75,mapbox_style=i)
    #orange is most affected area
    fig.show()

stamen-watercolor


In [74]:
#global
df = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/weekly_reviews.csv'))
list1 = df.week1.values.tolist()
minval = min(list1)
list1 = list(np.array(list1) - minval)
m = interp1d([0,max(list1)],[1,50])
circ_rad = m(list1)
typelist = ["stamen-watercolor"]

for i in typelist:
    print(i)
    fig = px.density_mapbox(df, lat="lat", lon="long",radius=circ_rad,zoom=2,mapbox_style=i)
    #orange is most affected area
    fig.show()

stamen-watercolor
